# About this notebook
This notebook is for batching 3D analysis on multiple images at once.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob

import pandas as pd

from iob_ia.utils.io_utils import (
    read_image,
)

## 1. Define some variables

In [13]:
# Define the path to the folder (use '/', and make sure that the path also ends with '/')
path_to_folder = "G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/"

# Image file type suffix
suffix = ".vsi"
# Channels that should be segmented (0-based indexing)
channels_to_segment = [1,2]
# In case metadata is missing (will be checked later...), specify the voxel size here
voxel_size = (1.46, 0.65, 0.65) # or None

# Filtering: a list of dictionaries (same order as channels_to_segment)
filters = [
    # First channel
    {
        "area": [430, 21600],
        ## FIXME: Need a way to make this dynamic...
        #"intensity_mean": [estimate_threshold("intensity_mean", properties) / 2, float("inf")],
        "intensity_mean": [250, float("inf")],
    },
    # Second channel
    {
        "area": [430, 21600],
        "intensity_mean": [150, float("inf")],
        "intensity_max": [None, 15000],
    }
]

In [4]:
# check the image paths
image_paths = glob(path_to_folder + "*" + suffix)
for p in image_paths:
    print(p)

G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01.vsi
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_25umdisk_01.vsi
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_30x_Nq027um_405_488_640_01.vsi


In [8]:
# TEMP test only on twice the first image
image_paths = [image_paths[0], image_paths[0]]
for p in image_paths:
    print(p)


G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01.vsi
G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01.vsi


In [14]:
# Check the images
img = None
all_channelsnames = []
all_voxelsizes = []
for p in image_paths:
    img = read_image(p)
    all_voxelsizes.append(img.physical_pixel_sizes)
    all_channelsnames.append(img.channel_names)
del img

channels_ok = True # check if images have all same number of channels
channels_equal = True # this is to check if the requested channels to segment is ok
voxels_ok = True # check for None in voxel sizes
voxels_equal = True # check that all voxel sizes are the same
n_channels = len(all_channelsnames[0])
vs = all_voxelsizes[0]
for i in range(len(image_paths)):
    # check that all images have the same number of channels
    if len(all_channelsnames[i]) != n_channels:
        channels_ok = False
    # Check that there are "enough channels"
    if max(channels_to_segment) + 1  > len(all_channelsnames[i]):
        channels_equal = False
    # Check that the voxel metadata is ok
    if None in all_voxelsizes[i]:
        voxels_ok = False
    # Check that the voxel sizes of all images is the same
    if vs[0] != all_voxelsizes[i][0] or vs[1] != all_voxelsizes[i][1] or vs[2] != all_voxelsizes[i][2]:
        voxels_equal = False

# check if voxel_size info has been supplied
if not voxels_ok and voxel_size is not None:
    print("Using manually defined voxel size")
    voxel_size = all_voxelsizes[0]
    voxels_ok = True
elif voxels_ok and voxel_size is not None:
    print("Using manually defined voxel size, even though metadata of images is fine")
    voxels_ok = True
elif voxels_ok and voxel_size is None:
    # use metadata from image
    voxel_size = all_voxelsizes[0]
    voxels_ok = True

if channels_ok and channels_equal and voxels_ok and voxels_equal:
    print("All good!")
else:
    print("There are some problems...:")
    if not channels_ok:
        print("- some images don't have the same number of channels")
    if not channels_equal:
        print("- some images do not have enough channels, for the channel numbers to segment you requested")
    if not voxels_ok:
        print("- some images do not have proper voxel sizes in the metadata")
    if not voxels_equal:
        print("- the images do not have all the same voxel size")

    image_names = []
    for p in image_paths:
        image_names.append(os.path.basename(p))
    check_sum = {
        "images": image_names,
        "channel names": all_channelsnames,
        "voxel sizes:": all_voxelsizes,
    }
    check_sum = pd.DataFrame.from_dict(check_sum)
    print(check_sum)


Using manually defined voxel size
All good!
